In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dataset_path = '/content/drive/My Drive/dataset'

In [4]:
import os
import numpy as np
from PIL import Image
from skimage.feature import hog
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [5]:
def load_images_from_folder(folder):
    images = []
    labels = []
    for class_folder in os.listdir(folder):
        class_path = os.path.join(folder, class_folder)
        if not os.path.isdir(class_path):
            continue
        for filename in os.listdir(class_path):
            img_path = os.path.join(class_path, filename)
            img = Image.open(img_path).convert('L')  # Convert to grayscale
            img = img.resize((64, 64))  # Resize to a fixed size

            # Extract HOG features
            img_array = np.array(img)
            hog_features = hog(img_array, pixels_per_cell=(8, 8), cells_per_block=(2, 2), feature_vector=True)

            images.append(hog_features)
            labels.append(class_folder)
    return images, labels

In [6]:
train_folder = os.path.join(dataset_path, 'train')
test_folder = os.path.join(dataset_path, 'test')
X_train, y_train = load_images_from_folder(train_folder)
X_test, y_test = load_images_from_folder(test_folder)

In [7]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [8]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [12]:
ann = MLPClassifier(hidden_layer_sizes=(100,),
                    max_iter=300,
                    solver='sgd',
                    learning_rate_init=0.01,
                    learning_rate='adaptive',
                    activation='logistic',  # Use sigmoid activation function
                    random_state=42)

ann.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', learning_rate='adaptive',
              learning_rate_init=0.01, max_iter=300, random_state=42,
              solver='sgd')

In [13]:
y_pred = ann.predict(X_test)

In [14]:
# KPIs
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [15]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1 Score: {f1:.2f}")

Accuracy: 60.71%
Precision: 60.89%
Recall: 60.71%
F1 Score: 0.61
